In [11]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, HTML
from pathlib import Path
from typing import *
import os


In [21]:
# Source ipyleaflets split screen docs

m = Map(center=(42.6824, 365.581), zoom=5)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-11-11")
left_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisAquaBands721CR, "2017-11-11")

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …